staple import

In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

EDA

**Column description**
1. Customer Number	 
A unique company identifier 
2. Campaign Period Sales 	 
Total dollar purchased during the campaign period 
3. Historical Sales Volume 	
Total historical dollar purchased
4. Date of First Purchase	
Date of first transaction
5. Number of Prior Year Transactions	
The number of transactions in the year prior to the promotion period
6. Do Not Direct Mail Solicit	
Does not allow promotional mail
7. Do Not Email	
Does not allow promotional email
8. Do Not Telemarket	
Does not allow promotional calls
9. Repurchase Method	
Method of last purchase prior to the campaign
10. Last Transaction Channel	
Sales channel of last purchase prior to the campaign
11. Desk	
Prior purchase of this item
12. Executive Chair	
Prior purchase of this item
13. Standard Chair	
Prior purchase of this item
14. Monitor	
Prior purchase of this item
15. Printer	
Prior purchase of this item
16. Computer	
Prior purchase of this item
17. Insurance	
Prior purchase of this item
18. Toner	
Prior purchase of this item
19. Office Supplies	
Prior purchase of this item
20. Number of Employees	
Purchased data on no. of employees
21. Language	
Primary language spoken

In [ ]:
df_temp = pd.read_excel (r'/kaggle/input/OfficeSupplyCampaignResultS7-23-19.xlsx', sheet_name='Campaign Results')
df_temp.head(3)

In [ ]:
df_temp.info()

refresh database

In [ ]:
df = df_temp

check duplicate record

In [ ]:
df.nunique()

Does Campaign Period Sales consist only the transaction directly to the campaign or both this campaign transaction and normal transaction 

Should we take Campaign Period Sales where >0 as participate and =0 as not participate

Or get a mean of the dollar purchased based on Number of Prior Year Transactions, and then take Campaign Period Sales minus mean where >mean as participate and =mean as not participate

In [ ]:
df['Campaign Period Sales'].describe()

In [ ]:
df.loc[df['Campaign Period Sales'] > 0, 'Campaign Period Sales'] = 1
df.loc[df['Campaign Period Sales'] == 0, 'Campaign Period Sales'] = 0
df['Campaign Period Sales'].value_counts()  

In [ ]:
df = df[df['Campaign Period Sales'] >= 0]
df['Campaign Period Sales'].value_counts()  

Data cleaning & mapping

In [ ]:
df = df.drop('Customer Number', axis = 1)

In [ ]:
df['Do Not Direct Mail Solicit'].value_counts()

In [ ]:
df['Do Not Email'].value_counts()    


In [ ]:
df['Do Not Telemarket'].value_counts()


In [ ]:
df['Repurchase Method'].value_counts()

In [ ]:
df['Last Transaction Channel'].value_counts()

In [ ]:
df['Last Transaction Channel'].isnull().sum()

In [ ]:
df['Desk'].value_counts()

There is a problem with Desk column, 1 extra line and 1 incorrect value

Find out the location of this adnormal to decide next action

Incorrect value will map later

In [ ]:
print(df[df['Desk']==911].index.values)

In [ ]:
df.tail(3)

In [ ]:
df = df[df.Desk != 911]
df.tail(3)

In [ ]:
df['Desk'].value_counts()

In [ ]:
df['Desk'] = df['Desk'].map( {'N': 0, 'Y': 1, 'YY': 1} )

In [ ]:
df['Desk'].value_counts()

In [ ]:
df['Executive Chair'].value_counts()

In [ ]:
df['Executive Chair'] = df['Executive Chair'].map( {'N': 0, 'Y': 1} )

In [ ]:
df['Executive Chair'].value_counts()

In [ ]:
df['Standard Chair'].value_counts()


In [ ]:
df['Standard Chair'] = df['Standard Chair'].map( {'N': 0, 'Y': 1} )
df['Standard Chair'].value_counts()

In [ ]:
df['Monitor'].value_counts()


In [ ]:
df['Monitor'] = df['Monitor'].map( {'N': 0, 'Y': 1} )
df['Monitor'].value_counts()

In [ ]:
df['Printer'].value_counts()


In [ ]:
df['Printer'] = df['Printer'].map( {'N': 0, 'Y': 1} )
df['Printer'].value_counts()

In [ ]:
df['Computer'].value_counts()


In [ ]:
df['Computer'] = df['Computer'].map( {'N': 0, 'Y': 1,'YY': 1} )
df['Computer'].value_counts()

In [ ]:
df['Insurance'].value_counts()


In [ ]:
df['Insurance'] = df['Insurance'].map( {'N': 0, 'Y': 1,'YY': 1} )
df['Insurance'].value_counts()

In [ ]:
df['Toner'].value_counts()


In [ ]:
df['Toner'] = df['Toner'].map( {'N': 0, 'Y': 1,'YY': 1} )
df['Toner'].value_counts()

In [ ]:
df['Office Supplies'].value_counts()


In [ ]:
df['Office Supplies'] = df['Office Supplies'].map( {'N': 0, 'Y': 1,'YY': 1} )
df['Office Supplies'].value_counts()

In [ ]:
df['Number of Employees'].value_counts()

blank value on numbers of employee. quantity are significant enough to not drop

In [ ]:
df['Number of Employees_copy'] = df['Number of Employees']
df['Number of Employees_copy'] = df['Number of Employees_copy'].replace(r'^\s*$', np.NaN, regex=True)
df['Number of Employees_copy'] = df['Number of Employees_copy'].astype(str)
df['Number of Employees_copy'] = df['Number of Employees_copy'].map({'1-5': 0, '6-10': 1, '11-50': 2, '51-100': 3, '101-500': 4, '500+': 5, 'nan': 6})
df['Number of Employees_copy'].value_counts()

In [ ]:
df['Number of Employees_copy']=df['Number of Employees_copy'].replace(6, df['Number of Employees_copy'].mean(), regex=True)
df['Number of Employees_copy'].value_counts()

In [ ]:
df = df.drop('Number of Employees_copy', axis = 1)

category mean suggest the blanks should be in between of category 11-50 and category 51-100

will merge these 3 category together

In [ ]:
#df['Number of Employees'] = df['Number of Employees'].map({'11-50': 11-100, '51-100': 11-100, r'^\s*$': 11-100})
df['Number of Employees'] = df['Number of Employees'].replace(r'^\s*$', np.nan, regex=True)
df['Number of Employees'] = df['Number of Employees'].astype(str)
df['Number of Employees'] = df['Number of Employees'].replace("11-50","11-100").replace("51-100","11-100").replace("nan","11-100")

In [ ]:
df['Number of Employees'].value_counts()

In [ ]:
df['Language'].value_counts()

group all the non english together and group nan into english

In [ ]:

List=df.Language.value_counts().index[1:].tolist()
mapping={}
for s in List:
    mapping[s]='Else'
df['Language']=df['Language'].map(lambda x: mapping[x] if x in mapping else x)
df['Language'] = df['Language'].map( {'English': 1,'Else': 0} )
df.Language.value_counts()
df.rename(columns={'Language': 'isEnglish'}, inplace=True)
df['isEnglish'] = df['isEnglish'].replace(np.nan,1 , regex=True)

In [ ]:
df['isEnglish'].value_counts()

In [ ]:
df.info()